In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import pandas as pd

import locale
from datetime import datetime, timedelta
import time

### Dev

In [2]:
driver = webdriver.Firefox()

In [3]:
driver.get('https://www.kn-online.de/themen/schleswig-holstein-veranstaltungen/') #URL stabil?

In [5]:
# TODO cookie dings reparieren
# TODO Benachrichtigungen ausschalten

try:
    # Wait for the button to be present and clickable
    wait = WebDriverWait(driver, 10)
    #button = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[@title='Einwilligen und weiter']")))
    button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button.cmp-button-primary[title='Einwilligen und weiter']")))
    # Click the button
    button.click()
    print("Button clicked successfully.")
except Exception as e:
    print(f"An error occurred: {e}")

An error occurred: Message: 
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:193:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:511:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:136:16



In [7]:
try:
    # Wait for the link to be present
    wait = WebDriverWait(driver, 10)
    
    # geht aber ggf nicht robust, wird sich zeigen
    #link_element = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "a.ContentTeaserstyled__Link-sc-1s3lhli-11.jMiOfI")))
    
    #robuster (wahrscheinlich)
    parent_div = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "div[data-testid='ContentList']")))
    link_element = parent_div.find_element(By.CSS_SELECTOR, "a")

    # Get the href attribute
    href = link_element.get_attribute('href')
    print(f"Collected href: {href}")
    
    # Visit the article
    driver.get(href)
    print("Navigated to the article successfully.")
except Exception as e:
    print(f"An error occurred: {e}")

Collected href: https://www.kn-online.de/schleswig-holstein/tipps-veranstaltungen-am-wochenende-in-schleswig-holstein-28-9-2024-und-29-9-2024-VVGPI5FNAVFQVPNN3IKPJPZLQM.html
Navigated to the article successfully.


In [8]:
# Extract events
events = []

# Locate the sections for the events
sections = driver.find_elements(By.CSS_SELECTOR, 'div.Articlestyled__ArticleBodyWrapper-sc-7y75gq-2')

for section in sections:
    date_elements = section.find_elements(By.CSS_SELECTOR, 'h2[data-testid="headline"]')
    for date_element in date_elements:
        date = date_element.text
        
        # Find the next sibling element which contains categories and text
        next_sibling = date_element.find_element(By.XPATH, 'following-sibling::ul')
        if next_sibling:
            category_elements = next_sibling.find_elements(By.XPATH, 'preceding-sibling::h2')
            for category_element in category_elements:
                category = category_element.text
                
                # Extract the list items under each category
                list_items = next_sibling.find_elements(By.CSS_SELECTOR, 'li.ListItemstyled__ListItem-sc-u7ur86-0')
                for item in list_items:
                    text = item.text
                    events.append({
                        'date': date,
                        'category': category,
                        'text': text
                    })

# Close the WebDriver
#driver.quit()

# Print the extracted events
for event in events:
    print(f"Date: {event['date']}")
    print(f"Category: {event['category']}")
    print(f"Text: {event['text']}")
    print("-" * 20)

NoSuchElementException: Message: Unable to locate element: following-sibling::ul; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:193:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:511:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:136:16


In [9]:
from selenium.common.exceptions import NoSuchElementException

# Extract events
events = []

# Locate the sections for the events
sections = driver.find_elements(By.CSS_SELECTOR, 'div.Articlestyled__ArticleBodyWrapper-sc-7y75gq-2')

for section in sections:
    date_elements = section.find_elements(By.CSS_SELECTOR, 'h2[data-testid="headline"]')
    for date_element in date_elements:
        date = date_element.text
        
        # Find the next sibling element which contains categories and text
        try:
            next_sibling = date_element.find_element(By.XPATH, 'following-sibling::*')
            while next_sibling and next_sibling.tag_name != 'ul':
                next_sibling = next_sibling.find_element(By.XPATH, 'following-sibling::*')
            
            if next_sibling:
                category_elements = next_sibling.find_elements(By.XPATH, 'preceding-sibling::h2')
                for category_element in category_elements:
                    category = category_element.text
                    
                    # Extract the list items under each category
                    list_items = next_sibling.find_elements(By.CSS_SELECTOR, 'li.ListItemstyled__ListItem-sc-u7ur86-0')
                    for item in list_items:
                        text = item.text
                        events.append({
                            'date': date,
                            'category': category,
                            'text': text
                        })
        except NoSuchElementException:
            continue

# Close the WebDriver
driver.quit()

# Print the extracted events
for event in events:
    print(f"Date: {event['date']}")
    print(f"Category: {event['category']}")
    print(f"Text: {event['text']}")
    print("-" * 20)

Date: Programm für Samstag, 12. Oktober 2024
Category: Programm für Samstag, 12. Oktober 2024
Text: Max Raabe in der Wunderino-Arena in Kiel: Unter dem Titel „Wer hat hier schlechte Laune?“ tritt Max Raabe am 12. Oktober ab 20 Uhr gemeinsam mit dem Palastorchester in der Wunderino-Arena (Europaplatz 1) in Kiel auf. Raabe mischt Eigenkompositionen mit Originalarrangements der 20er- und 30er-Jahre. Unter vielem anderen ist Raabe für das Titelstück der aktuellen „Babylon Berlin“-Staffel „Ein Tag wie Gold“ bekannt, welches er mit Annette Humpe geschrieben hat. Tickets gibt es ab 69,95 Euro unter anderem online.
--------------------
Date: Programm für Samstag, 12. Oktober 2024
Category: Programm für Samstag, 12. Oktober 2024
Text: Rock Tales im Kulturforum in Kiel: Geschichten aus der Rock-Vergangenheit gespickt mit dem richtigen Soundtrack wollen Jürgen Rau und Richard Rossbach am Samstag ab 20 Uhr im Kulturforum (Andreas-Gayk-Straße 31) zum Leben erwecken. Während Rau von Begegnungen mit 

In [10]:
pd.DataFrame(events)

,date,category,text
0,"Programm für Samstag, 12. Oktober 2024","Programm für Samstag, 12. Oktober 2024",Max Raabe in der Wunderino-Arena in Kiel: Unte...
1,"Programm für Samstag, 12. Oktober 2024","Programm für Samstag, 12. Oktober 2024",Rock Tales im Kulturforum in Kiel: Geschichten...
2,"Programm für Samstag, 12. Oktober 2024","Programm für Samstag, 12. Oktober 2024",Cumbia im Spieker in Eckernförde: Getanzt werd...
3,"Programm für Samstag, 12. Oktober 2024",Musik und Konzerte,Max Raabe in der Wunderino-Arena in Kiel: Unte...
4,"Programm für Samstag, 12. Oktober 2024",Musik und Konzerte,Rock Tales im Kulturforum in Kiel: Geschichten...
...,...,...,...
127,Sonstiges,Kunst & Kultur,Apfelfest in Kiel-Ellerbek: Das Permakulturzen...
128,Sonstiges,Familienprogramm,„Zebra-Tag“ in Altenholz: Am 13. Oktober lädt ...
129,Sonstiges,Familienprogramm,Apfelfest in Kiel-Ellerbek: Das Permakulturzen...
130,Sonstiges,Sonstiges,„Zebra-Tag“ in Altenholz: Am 13. Oktober lädt ...
